In [1]:
import pandas as pd
import numpy as np
import requests
import psycopg2
import os

In [2]:
filename = 'DataSet-Troconiz-E1.csv'
url = "DataSet-Troconiz-E1.csv"
dataset = pd.read_csv(url)

El data set se basa en un conjunto de datos de IBM Telco Customer Churn.


La compañía JB Link, una pequeña empresa de telecomunicaciones en California, enfrenta una alta tasa de churn de clientes lo que significa que muchos clientes están cancelando sus servicios o dejando de usarlos y ha creado un equipo de trabajo, que incluye un equipo de ciencia de datos, para desarrollar una estrategia de retención de clientes. 

El equipo de ciencia de datos se encargará de obtener información de los datos, desarrollar un modelo de aprendizaje automático para predecir la churn de clientes y prescribir acciones personalizadas para retener a cada uno de esos clientes. Los ejecutivos esperan que el proyecto ahorre mucho dinero a la compañía y la haga crecer nuevamente.


Muestreo de datos de todas las columnas:

In [3]:
pd.set_option('display.max_columns', 50)
dataset.head()

,Customer ID,Referred a Friend,Number of Referrals,Tenure in Months,Offer,Phone Service,Avg Monthly Long Distance Charges,Multiple Lines,Internet Service,Internet Type,Avg Monthly GB Download,Online Security,Online Backup,Device Protection Plan,Premium Tech Support,Streaming TV,Streaming Movies,Streaming Music,Unlimited Data,Contract,Paperless Billing,Payment Method,Monthly Charge,Total Regular Charges,Total Refunds,Total Extra Data Charges,Total Long Distance Charges,Gender,Age,Under 30,Senior Citizen,Married,Dependents,Number of Dependents,City,Zip Code,Latitude,Longitude,Population,Churn Value,CLTV,Churn Category,Churn Reason,Total Customer Svc Requests,Product/Service Issues Reported,Customer Satisfaction
0,8779-QRDMV,No,0,1,None,No,0.00,No,Yes,Fiber Optic,9,No,No,Yes,No,No,Yes,No,No,Month-to-Month,Yes,Bank Withdrawal,41.236,39.65,0.00,0.0,0.00,Male,78,No,Yes,No,No,0,Los Angeles,90022,34.023810,-118.156582,68701,1,5433,Competitor,Competitor offered more data,5,0,NaN
1,7495-OOKFY,Yes,1,8,Offer E,Yes,48.85,Yes,Yes,Cable,19,No,Yes,No,No,No,No,No,No,Month-to-Month,Yes,Credit Card,83.876,633.30,0.00,120.0,390.80,Female,74,No,Yes,Yes,Yes,1,Los Angeles,90063,34.044271,-118.185237,55668,1,5302,Competitor,Competitor made better offer,5,0,NaN
2,1658-BYGOY,No,0,18,Offer D,Yes,11.33,Yes,Yes,Fiber Optic,57,No,No,No,No,Yes,Yes,Yes,Yes,Month-to-Month,Yes,Bank Withdrawal,99.268,1752.55,45.61,0.0,203.94,Male,71,No,Yes,No,Yes,3,Los Angeles,90065,34.108833,-118.229715,47534,1,3179,Competitor,Competitor made better offer,1,0,NaN
3,4598-XLKNJ,Yes,1,25,Offer C,Yes,19.76,No,Yes,Fiber Optic,13,No,Yes,Yes,No,Yes,Yes,No,No,Month-to-Month,Yes,Bank Withdrawal,102.440,2514.50,13.43,327.0,494.00,Female,78,No,Yes,Yes,Yes,1,Inglewood,90303,33.936291,-118.332639,27778,1,5337,Dissatisfaction,Limited range of services,1,1,2.0
4,4846-WHAFZ,Yes,1,37,Offer C,Yes,6.33,Yes,Yes,Cable,15,No,No,No,No,No,No,No,No,Month-to-Month,Yes,Bank Withdrawal,79.560,2868.15,0.00,430.0,234.21,Female,80,No,Yes,Yes,Yes,1,Whittier,90602,33.972119,-118.020188,26265,1,2793,Price,Extra data charges,1,0,2.0


Informacion referente a las columnas 

In [4]:
dataset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7043 entries, 0 to 7042
Data columns (total 46 columns):
 #   Column                             Non-Null Count  Dtype  
---  ------                             --------------  -----  
 0   Customer ID                        7043 non-null   object 
 1   Referred a Friend                  7043 non-null   object 
 2   Number of Referrals                7043 non-null   int64  
 3   Tenure in Months                   7043 non-null   int64  
 4   Offer                              7043 non-null   object 
 5   Phone Service                      7043 non-null   object 
 6   Avg Monthly Long Distance Charges  7043 non-null   float64
 7   Multiple Lines                     7043 non-null   object 
 8   Internet Service                   7043 non-null   object 
 9   Internet Type                      7043 non-null   object 
 10  Avg Monthly GB Download            7043 non-null   int64  
 11  Online Security                    7043 non-null   objec

In [5]:
dataset.describe()

,Number of Referrals,Tenure in Months,Avg Monthly Long Distance Charges,Avg Monthly GB Download,Monthly Charge,Total Regular Charges,Total Refunds,Total Extra Data Charges,Total Long Distance Charges,Age,Number of Dependents,Zip Code,Latitude,Longitude,Population,Churn Value,CLTV,Total Customer Svc Requests,Product/Service Issues Reported,Customer Satisfaction
count,7043.000000,7043.000000,7043.000000,7043.000000,7043.000000,7043.000000,7043.000000,7043.000000,7043.000000,7043.000000,7043.000000,7043.000000,7043.000000,7043.000000,7043.000000,7043.000000,7043.000000,7043.000000,7043.000000,1834.000000
mean,1.951867,32.386767,22.958954,21.110890,65.538800,2280.381264,1.962182,278.499225,749.099262,46.509726,0.468692,93486.070567,36.197455,-119.756684,22139.603294,0.265370,4400.295755,1.338776,0.308107,3.005453
std,3.001199,24.542061,15.448113,20.948471,30.606805,2266.220462,7.902614,685.039625,846.660055,16.750352,0.962802,1856.767505,2.468929,2.154425,21152.392837,0.441561,1183.057152,1.430471,0.717514,1.256938
min,0.000000,1.000000,0.000000,0.000000,18.250000,18.800000,0.000000,0.000000,0.000000,19.000000,0.000000,90001.000000,32.555828,-124.301372,11.000000,0.000000,2003.000000,0.000000,0.000000,1.000000
25%,0.000000,9.000000,9.210000,3.000000,35.890000,400.150000,0.000000,0.000000,70.545000,32.000000,0.000000,92101.000000,33.990646,-121.788090,2344.000000,0.000000,3469.000000,0.000000,0.000000,2.000000
50%,0.000000,29.000000,22.890000,17.000000,71.968000,1394.550000,0.000000,0.000000,401.440000,46.000000,0.000000,93518.000000,36.205465,-119.595293,17554.000000,0.000000,4527.000000,1.000000,0.000000,3.000000
75%,3.000000,55.000000,36.395000,28.000000,90.650000,3786.600000,0.000000,182.620000,1191.100000,60.000000,0.000000,95329.000000,38.161321,-117.969795,36125.000000,1.000000,5380.500000,2.000000,0.000000,4.000000
max,11.000000,72.000000,49.990000,94.000000,123.084000,8684.800000,49.790000,6477.000000,3564.720000,80.000000,9.000000,96150.000000,41.962127,-114.192901,105285.000000,1.000000,6500.000000,9.000000,6.000000,5.000000


# API

Para poder hacer el uso de la api se eliminaran las columnas del data frame columns_to_drop = ['City', 'Zip Code', 'Latitude', 'Longitude', 'Population', 'Gender', 'Age', 'Under 30'] y se traeran estos datos aleatoreamente de una API

In [6]:

# Lista de columnas a eliminar
columns_to_drop = ['City', 'Zip Code', 'Latitude', 'Longitude', 'Population', 'Gender', 'Age', 'Under 30', 'Customer ID']

# Elimina las columnas
dataset = dataset.drop(columns=columns_to_drop)



In [7]:
dataset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7043 entries, 0 to 7042
Data columns (total 37 columns):
 #   Column                             Non-Null Count  Dtype  
---  ------                             --------------  -----  
 0   Referred a Friend                  7043 non-null   object 
 1   Number of Referrals                7043 non-null   int64  
 2   Tenure in Months                   7043 non-null   int64  
 3   Offer                              7043 non-null   object 
 4   Phone Service                      7043 non-null   object 
 5   Avg Monthly Long Distance Charges  7043 non-null   float64
 6   Multiple Lines                     7043 non-null   object 
 7   Internet Service                   7043 non-null   object 
 8   Internet Type                      7043 non-null   object 
 9   Avg Monthly GB Download            7043 non-null   int64  
 10  Online Security                    7043 non-null   object 
 11  Online Backup                      7043 non-null   objec

Se utilizara una API para obtener los datos de los clientes y comentarios:

In [8]:

base_url = 'https://randomuser.me/api/'
params = {'inc': 'gender,nat,phone,email,location,name,id', 'results': 5000}

new_dataset = pd.DataFrame()

#iterar 2 veces para obtener los 7043 registros 
#se le agrega un while para solo llenar los 7043
while len(new_dataset) < 7043:
    response = requests.get(base_url, params=params)
    data = response.json()
    api_df = pd.json_normalize(data['results'])

    # Concatena los datos de la API con tu DataFrame existente
    new_dataset = pd.concat([new_dataset, api_df], ignore_index=True)

    # Si la longitud del DataFrame supera 7043, se trunca a 7043
    if len(new_dataset) > 7043:
        new_dataset = new_dataset.iloc[:7043]


In [9]:
new_dataset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7043 entries, 0 to 7042
Data columns (total 19 columns):
 #   Column                          Non-Null Count  Dtype 
---  ------                          --------------  ----- 
 0   gender                          7043 non-null   object
 1   email                           7043 non-null   object
 2   phone                           7043 non-null   object
 3   nat                             7043 non-null   object
 4   name.title                      7043 non-null   object
 5   name.first                      7043 non-null   object
 6   name.last                       7043 non-null   object
 7   location.street.number          7043 non-null   int64 
 8   location.street.name            7043 non-null   object
 9   location.city                   7043 non-null   object
 10  location.state                  7043 non-null   object
 11  location.country                7043 non-null   object
 12  location.postcode               7043 non-null   

Se analizan los datos para ver valores duplicados de los id 

In [10]:
duplicated_ids = new_dataset["id.value"].duplicated()
print("Number of duplicated IDs:", duplicated_ids.sum())

Number of duplicated IDs: 1384


Se dropean los valores duplicados

In [11]:
new_dataset = new_dataset.drop_duplicates(subset='id.value')

In [12]:
new_dataset.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 5659 entries, 0 to 7042
Data columns (total 19 columns):
 #   Column                          Non-Null Count  Dtype 
---  ------                          --------------  ----- 
 0   gender                          5659 non-null   object
 1   email                           5659 non-null   object
 2   phone                           5659 non-null   object
 3   nat                             5659 non-null   object
 4   name.title                      5659 non-null   object
 5   name.first                      5659 non-null   object
 6   name.last                       5659 non-null   object
 7   location.street.number          5659 non-null   int64 
 8   location.street.name            5659 non-null   object
 9   location.city                   5659 non-null   object
 10  location.state                  5659 non-null   object
 11  location.country                5659 non-null   object
 12  location.postcode               5659 non-null   

Ahora procedemos al mergue de los datos creaandole a ambas una columna index

In [13]:
dataset['index'] = range(1, len(dataset) + 1)
new_dataset['index'] = range(1, len(new_dataset) + 1)

In [14]:
merged_df = pd.merge(dataset, new_dataset, on='index')

In [15]:
dataset = merged_df.copy()


 Se igualan los data set para trabajar con el "dataset" solo con los valores que tienen id y datos 

In [16]:
dataset = merged_df.copy()


In [17]:
dataset.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 5659 entries, 0 to 5658
Data columns (total 57 columns):
 #   Column                             Non-Null Count  Dtype  
---  ------                             --------------  -----  
 0   Referred a Friend                  5659 non-null   object 
 1   Number of Referrals                5659 non-null   int64  
 2   Tenure in Months                   5659 non-null   int64  
 3   Offer                              5659 non-null   object 
 4   Phone Service                      5659 non-null   object 
 5   Avg Monthly Long Distance Charges  5659 non-null   float64
 6   Multiple Lines                     5659 non-null   object 
 7   Internet Service                   5659 non-null   object 
 8   Internet Type                      5659 non-null   object 
 9   Avg Monthly GB Download            5659 non-null   int64  
 10  Online Security                    5659 non-null   object 
 11  Online Backup                      5659 non-null   objec

# Subir datos a AWS  

In [38]:
# Establecer los parámetros de conexión
dbname = 'data-engineer-database'
host = 'data-engineer-cluster.cyhh5bfevlmn.us-east-1.redshift.amazonaws.com'
port = '5439'
user = 'jtroconiz_coderhouse'
password = '4n170E04Fd'
#password = os.environ['REDSHIFT_PASSWORD'] no me funciono la escondinda de la password 


In [39]:
conn = psycopg2.connect(
    dbname=dbname,
    host=host,
    port=port,
    user=user,
    password=password
)
cur = conn.cursor()

In [40]:
# Nombre de la tabla y esquema en Redshift
tabla_redshift = 'CHURN'
esquema_redshift = 'jtroconiz_coderhouse'


In [41]:
# Sentencia SQL para crear la tabla
create_table_query = '''
    CREATE TABLE IF NOT EXISTS jtroconiz_coderhouse.CHURN (
        "Referred a Friend" varchar(255),
        "Number of Referrals" integer,
        "Tenure in Months" integer,
        "Offer" varchar(255),
        "Phone Service" varchar(255),
        "Avg Monthly Long Distance Charges" float,
        "Multiple Lines" varchar(255),
        "Internet Service" varchar(255),
        "Internet Type" varchar(255),
        "Avg Monthly GB Download" integer,
        "Online Security" varchar(255),
        "Online Backup" varchar(255),
        "Device Protection Plan" varchar(255),
        "Premium Tech Support" varchar(255),
        "Streaming TV" varchar(255),
        "Streaming Movies" varchar(255),
        "Streaming Music" varchar(255),
        "Unlimited Data" varchar(255),
        "Contract" varchar(255),
        "Paperless Billing" varchar(255),
        "Payment Method" varchar(255),
        "Monthly Charge" float,
        "Total Regular Charges" float,
        "Total Refunds" float,
        "Total Extra Data Charges" float,
        "Total Long Distance Charges" float,
        "Senior Citizen" varchar(255),
        "Married" varchar(255),
        "Dependents" varchar(255),
        "Number of Dependents" integer,
        "Churn Value" integer,
        "CLTV" integer,
        "Churn Category" varchar(255),
        "Churn Reason" varchar(255),
        "Total Customer Svc Requests" integer,
        "Product/Service Issues Reported" integer,
        "Customer Satisfaction" float,
        "index" integer,
        "gender" varchar(255),
        "email" varchar(255),
        "phone" varchar(255),
        "nat" varchar(255),
        "name.title" varchar(255),
        "name.first" varchar(255),
        "name.last" varchar(255),
        "location.street.number" integer,
        "location.street.name" varchar(255),
        "location.city" varchar(255),
        "location.state" varchar(255),
        "location.country" varchar(255),
        "location.postcode" varchar(255),
        "location.coordinates.latitude" varchar(255),
        "location.coordinates.longitude" varchar(255),
        "location.timezone.offset" varchar(255),
        "location.timezone.description" varchar(255),
        "id.name" varchar(255),
        "id.value" varchar(255)
    );
'''

In [42]:
cur.execute(create_table_query)
conn.commit()

# Se verifica la creacion de la tabla y las columnas y se continua con la suba de informacion 

In [43]:
# Obtener los primeros 10 registros del dataset no repetidos
subset_dataset = dataset.head(30)

# Insertar los datos en la tabla de Redshift si no existen ya
for index, row in subset_dataset.iterrows():
    values = ', '.join(['%s'] * len(row))
    columns = [
        "\"Referred a Friend\"", "\"Number of Referrals\"", "\"Tenure in Months\"", "\"Offer\"", "\"Phone Service\"",
        "\"Avg Monthly Long Distance Charges\"", "\"Multiple Lines\"", "\"Internet Service\"", "\"Internet Type\"",
        "\"Avg Monthly GB Download\"", "\"Online Security\"", "\"Online Backup\"", "\"Device Protection Plan\"",
        "\"Premium Tech Support\"", "\"Streaming TV\"", "\"Streaming Movies\"", "\"Streaming Music\"",
        "\"Unlimited Data\"", "\"Contract\"", "\"Paperless Billing\"", "\"Payment Method\"",
        "\"Monthly Charge\"", "\"Total Regular Charges\"", "\"Total Refunds\"", "\"Total Extra Data Charges\"",
        "\"Total Long Distance Charges\"", "\"Senior Citizen\"", "\"Married\"", "\"Dependents\"", "\"Number of Dependents\"",
        "\"Churn Value\"", "\"CLTV\"", "\"Churn Category\"", "\"Churn Reason\"", "\"Total Customer Svc Requests\"",
        "\"Product/Service Issues Reported\"", "\"Customer Satisfaction\"", "\"index\"", "\"gender\"", "\"email\"",
        "\"phone\"", "\"nat\"", "\"name.title\"", "\"name.first\"", "\"name.last\"", "\"location.street.number\"",
        "\"location.street.name\"", "\"location.city\"", "\"location.state\"", "\"location.country\"",
        "\"location.postcode\"", "\"location.coordinates.latitude\"", "\"location.coordinates.longitude\"",
        "\"location.timezone.offset\"", "\"location.timezone.description\"", "\"id.name\"", "\"id.value\""
    ]

    columns_str = ', '.join(columns)
    insert_query = f"""
        INSERT INTO {esquema_redshift}.{tabla_redshift} ({columns_str})
        SELECT {values}
        WHERE NOT EXISTS (
            SELECT 1 FROM {esquema_redshift}.{tabla_redshift}
            WHERE "id.value" = %s
        )
    """
    cur.execute(insert_query, (tuple(row) + (row["id.value"],)))  # Agregar el valor de "id.value" para la cláusula WHERE

# Confirmar los cambios y cerrar la conexión
conn.commit()
cur.close()
conn.close()
